Import

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math as math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import sklearn
from sklearn import svm, preprocessing 
import warnings

Reading Data

In [6]:
data = pd.read_csv("../../data_sets/Volumetric_features.csv")

df = pd.DataFrame(data, columns = data.columns[:-1])
df = df.drop(['S.No','Age'], axis=1)

df.head()
df.describe()


scaler = StandardScaler()
x = scaler.fit_transform(df)
n = 20
pca = PCA(n_components=n)
pca_dataF = pca.fit_transform(x)

labels = df.columns.values.tolist()
label_index = [np.abs(pca.components_[i]).argmax() for i in range(n)]
columns = [labels[label_index[i]] for i in range(n)]

pca_df = pd.DataFrame(data=pca_dataF, columns=columns)
print(pca_df.head)
pca_df.head()

<bound method NDFrame.head of       rh_MeanThickness_thickness  CerebralWhiteMatterVol  \
0                       1.754401                1.293660   
1                       1.417516                1.506792   
2                       2.060537                1.356491   
3                       4.321472                1.316559   
4                       3.432616                1.645477   
...                          ...                     ...   
4221                    3.508241                2.349696   
4222                    4.445945               -2.409496   
4223                    8.016491                2.326574   
4224                   -0.596625                2.547034   
4225                   -2.307309               -3.481650   

      Left-Lateral-Ventricle  lh_pericalcarine_thickness  SurfaceHoles  \
0                   1.400014                   -1.478341     -1.846886   
1                   1.653995                   -0.613379     -1.383478   
2                   1.48903

,rh_MeanThickness_thickness,CerebralWhiteMatterVol,Left-Lateral-Ventricle,lh_pericalcarine_thickness,SurfaceHoles,CC_Posterior,rh_caudalanteriorcingulate_thickness,CC_Posterior,Right-Caudate,lh_parahippocampal_thickness,MaskVol-to-eTIV,Brain-Stem,Left-vessel,Right-vessel,non-WM-hypointensities,rh_isthmuscingulate_thickness,5th-Ventricle,5th-Ventricle,5th-Ventricle,5th-Ventricle
0,1.754401,1.293660,1.400014,-1.478341,-1.846886,2.544577,-1.132540,-0.396192,-0.386675,1.583273,-0.215873,0.204923,-1.697293,0.726855,-0.432085,0.423524,0.148992,-0.442516,0.298721,0.141646
1,1.417516,1.506792,1.653995,-0.613379,-1.383478,2.366964,-1.434173,-0.837984,-0.786456,1.735877,0.198549,0.410625,-1.577886,0.365404,-0.886260,0.131859,-0.358089,-0.443751,0.799788,0.010543
2,2.060537,1.356491,1.489035,-1.054925,-1.173713,2.164942,-2.157873,-0.800740,-1.133369,1.485384,0.076729,0.353749,-1.042033,0.829543,-0.888619,0.433455,-0.552787,-0.196933,0.610856,-0.470847
3,4.321472,1.316559,1.100140,-0.239404,-1.373375,2.413662,-1.142020,-1.549904,-0.847564,1.341931,0.295439,-0.309571,-1.485291,0.704329,-0.311530,0.123054,-0.255345,-0.197710,0.585629,-0.332968
4,3.432616,1.645477,2.126583,-0.803944,-1.321405,3.075919,-1.200196,-0.804217,-0.563766,1.545778,-0.076668,0.445481,-1.512658,1.425505,-0.855445,-0.023213,0.193964,-0.385681,0.889023,0.217353


Training Data

In [7]:
# Split for validation --> train, val, test = 80/15/5

#x_train, x_validation, y_train, y_validation = train_test_split(data.drop(['S.No','Age'], axis=1), data['Age'], test_size=0.20, random_state=33)

x_train, x_validation, y_train, y_validation = train_test_split(pca_df, data['Age'], test_size=0.20, random_state=33)

# (20% of total dataset -> 75% validation = 15% total, 25% validation = 5% total

x_val, x_test, y_val, y_test = train_test_split(x_validation, y_validation, test_size=0.25, random_state=33)

print("x_train shape is:",x_train.shape)
print("y_train shape is:",y_train.shape, "\n")
print("x_val shape is:",x_val.shape)
print("y_val shape is:",y_val.shape, "\n")
print("x_test shape is:",x_test.shape)
print("y_test shape is:",y_test.shape)

x_train shape is: (3380, 20)
y_train shape is: (3380,) 

x_val shape is: (634, 20)
y_val shape is: (634,) 

x_test shape is: (212, 20)
y_test shape is: (212,)


Lasso Regression

In [8]:
las_reg = linear_model.Lasso()
las_reg.fit(x_train,y_train)
y_pred = las_reg.predict(x_test)
print("accuracy: "+ str(las_reg.score(x_test,y_test)*100) + "%")
print("Mean absolute error: {}".format(mean_absolute_error(y_test,y_pred)))
print("Mean squared error: {}".format(mean_squared_error(y_test,y_pred)))
print("Root mean squared error: {}".format(math.sqrt(mean_squared_error(y_test,y_pred))))
R2 = r2_score(y_test,y_pred)
print('R Squared: {}'.format(R2))
n=x_test.shape[0]
p=x_test.shape[1] - 1

adj_rsquared = 1 - (1 - R2) * ((n - 1)/(n-p-1))
print('Adjusted R Squared: {}'.format(adj_rsquared))

warnings.filterwarnings('ignore')
warnings.warn('Delftstack')

accuracy: 76.15004271788311%
Mean absolute error: 7.953830016324598
Mean squared error: 93.5129952438426
Root mean squared error: 9.670211747621796
R Squared: 0.761500427178831
Adjusted R Squared: 0.7378989069517362
